In [1]:
!pip install -qU \
  langchain-core \
  langchain-google-genai \
  langchain-community \
  scikit-image


In [2]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

Enter API key for Google Gemini: ········


In [3]:
from langchain_google_genai import GoogleGenerativeAI


llm = GoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0) 

## Conversation Buffer Memory

In [4]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

/tmp/ipykernel_2375/1448044083.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [5]:
memory.save_context(
    {"input": "Hi, my names in James"},
    {"output": "Hey James, what's up? I'm an AI model called Zeta."}
)
memory.save_context(
    {"input": "I am researching different types of conversational memory."},
    {"output": "That's interesting, what are some examples?"}
)
memory.save_context(
    {"input": "I am researching ConversationBufferMemory and ConversationBufferWindowMemory"},
    {"output": "That's interesting, what's thed difference"}
)
memory.save_context(
    {"input": "Buffer memory just store the enitre converation, right?"},
    {"output": "Thats makes sense, what about ConversationBufferWindowMemory"}
)
memory.save_context(
    {"input": "Buffer window memory stores last k messages, dropping the rest."},
    {"output": "Very cool!"}
)


In [6]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my names in James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  HumanMessage(conte

In [7]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)


/tmp/ipykernel_2375/2943370833.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(


In [8]:
chain.invoke({"input": "What is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my names in James', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}

{'input': 'What is my name again?',
 'history': [HumanMessage(content='Hi, my names in James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, respons

## ConversationBufferMemory with RunnableWithMessageHistory

In [9]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "You are a helpful assistant called Zeta."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}")   
])


In [10]:
pipeline = (
    { 
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
    }
    | prompt_template
    | llm
)

In [11]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

In [12]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [13]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123"}
)

"Hello James, it's nice to meet you. My name is Zeta. How can I help you today?"

In [14]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123"}
)

'Your name is James.'

## ConversationBufferWindowMemory

In [15]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=4, return_messages=True)

/tmp/ipykernel_2375/3216785012.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=4, return_messages=True)


In [16]:
memory.save_context(
    {"input": "Hi, my names in James"},
    {"output": "Hey James, what's up? I'm an AI model called Zeta."}
)
memory.save_context(
    {"input": "I am researching different types of conversational memory."},
    {"output": "That's interesting, what are some examples?"}
)
memory.save_context(
    {"input": "I am researching ConversationBufferMemory and ConversationBufferWindowMemory"},
    {"output": "That's interesting, what's thed difference"}
)
memory.save_context(
    {"input": "Buffer memory just store the enitre converation, right?"},
    {"output": "Thats makes sense, what about ConversationBufferWindowMemory"}
)
memory.save_context(
    {"input": "Buffer window memory stores last k messages, dropping the rest."},
    {"output": "Very cool!"}
)

In [17]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]}

In [18]:
chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

In [19]:
chain.invoke({"input": "What is my name again"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}), HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}), AIMessage(content='Thats makes sense, what about Conversat

{'input': 'What is my name again',
 'history': [HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kwargs={}, respon

In [20]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)
    
    
    def __init__(self, k:int):
        super().__init__(k=k)
        print(f"Initializing BufferWindowMessageHistory with k={k}")
        
    
    def add_messages(self, messages: list[BaseMessage]) -> None:
        """
        Add messages to the history, removing any messages beyond
        the last `k` messages
        """
        
        self.messages.extend(messages)
        self.messages = self.messages[-self.k:]
        
        
    def clear(self) -> None:
        """
        clear the history
        """
        self.messages = []       
        
        

In [21]:
chat_map = {}
def get_chat_window_history(session_id: str, k: int = 4) -> BufferWindowMessageHistory:
    if session_id not in chat_map:
        chat_map[session_id] = BufferWindowMessageHistory(k=k)
    
    return chat_map[session_id]

In [22]:
from langchain_core.runnables import ConfigurableFieldSpec

pipeline_cwbm = (
    { 
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
    }
    | prompt_template
    | llm
)
 

pipeline_with_history_cwbm = RunnableWithMessageHistory(
    pipeline_cwbm,
    get_session_history = get_chat_window_history,
    input_messages_key = "query",
    history_messages_key = "history",
    history_factory_config = [
        ConfigurableFieldSpec(
            id = "session_id",
            annotation = str,
            name = "Session ID",
            description = "The session ID to use for the chat history",
            default = "id_default"
        ),
        ConfigurableFieldSpec(
            id = "k",
            annotation = int,
            name = "k",
            description = "The number of messages to keep in the history",
            default = 4
        )
    ]
    
)

In [23]:
pipeline_with_history_cwbm.invoke(
    {
        "query": "my name is James"  
    },
    config = {
        "configurable": {
            "session_id" : "id_k3",
            "k" : 3
        }
    }

)

Initializing BufferWindowMessageHistory with k=3


"Okay, James. It's nice to meet you. How can I help you today?"

In [24]:
pipeline_with_history_cwbm.invoke(
    {
        "query": "I am learning langchain"  
    },
    config = {
        "configurable": {
            "session_id" : "id_k3",
            "k" : 3
        }
    }

)

"AI: That's great, James! Langchain is a powerful framework. What are you hoping to learn about it? Are you just starting out, or do you have some specific questions or goals in mind?\n\nFor example, are you interested in:\n\n*   **Understanding the core concepts?** (e.g., LLMs, Prompts, Chains, Agents, Memory)\n*   **Building a specific type of application?** (e.g., Question Answering, Chatbots, Document Summarization)\n*   **Working with a particular LLM?** (e.g., OpenAI, Hugging Face, Cohere)\n*   **Troubleshooting a specific issue?**\n\nThe more information you give me, the better I can assist you. Let me know what you'd like to focus on!"

In [25]:
pipeline_with_history_cwbm.invoke(
    {
        "query": "My plan is to learn Langgraph and Google A2A, which one should I go"  
    },
    config = {
        "configurable": {
            "session_id" : "id_k3",
            "k" : 3
        }
    }

)

'AI: Okay, James, that\'s a great plan! Both LangGraph and Google\'s A2A (Agent-to-Agent) communication are exciting areas. To help you decide where to start, let\'s break down the key differences and considerations:\n\n**LangGraph:**\n\n*   **Focus:** Orchestration and building complex, stateful, multi-actor workflows using LLMs. Think of it as a way to define a graph-like structure where different LLM-powered "nodes" (agents, tools, or functions) interact and pass information to each other.\n*   **Key Features:**\n    *   **Graph-based structure:**  Define the flow of information and control between different components.\n    *   **State management:**  Maintain state across multiple steps in the workflow.\n    *   **Conditional routing:**  Route the flow based on the output of previous steps.\n    *   **Integration with LangChain:**  Built on top of LangChain, so you can leverage existing LangChain components.\n*   **Use Cases:**\n    *   Complex agent interactions (e.g., a sales age

In [26]:
chat_map["id_k3"].messages[2]

AIMessage(content='AI: Okay, James, that\'s a great plan! Both LangGraph and Google\'s A2A (Agent-to-Agent) communication are exciting areas. To help you decide where to start, let\'s break down the key differences and considerations:\n\n**LangGraph:**\n\n*   **Focus:** Orchestration and building complex, stateful, multi-actor workflows using LLMs. Think of it as a way to define a graph-like structure where different LLM-powered "nodes" (agents, tools, or functions) interact and pass information to each other.\n*   **Key Features:**\n    *   **Graph-based structure:**  Define the flow of information and control between different components.\n    *   **State management:**  Maintain state across multiple steps in the workflow.\n    *   **Conditional routing:**  Route the flow based on the output of previous steps.\n    *   **Integration with LangChain:**  Built on top of LangChain, so you can leverage existing LangChain components.\n*   **Use Cases:**\n    *   Complex agent interactions 

## Converasation Summary Memory

In [27]:
from langchain.memory import ConversationSummaryMemory

summary_memory = ConversationSummaryMemory(llm = llm)

/tmp/ipykernel_2375/3825381965.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(llm = llm)


In [28]:
summary_chain = ConversationChain(
    llm = llm,
    memory = summary_memory,
    verbose = True
)

In [30]:
summary_chain.invoke({"input": "hello there my name in Muhammad"})
summary_chain.invoke({"input": "I am learning chain and exploring how it can be use"})
summary_chain.invoke({"input": "I am looking into ConversationBufferMemory and ConversationWindowBufferMemory"})
summary_chain.invoke({"input": "BufferMemory stores the entire conversation"})
summary_chain.invoke({"input": "BufferWindowMemory stores last k messages and dropped previous messages"})





> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces himself as Muhammad. The AI greets Muhammad, explains it doesn't have a name in the human sense but can be called AI, and describes itself as a large language model trained by Google, capable of communicating and generating human-like text. It then asks how it can help Muhammad. Muhammad says he is learning about "chain" and exploring its uses. The AI responds enthusiastically, clarifies that "chain" could refer to blockchain technology, Markov chains, or supply chain management, and provides a brief overview of each, including potential applications. It then asks Muhammad to specify which type of "chain" he is most interested in to provide more re


> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces himself as Muhammad. The AI greets Muhammad, explains it doesn't have a name in the human sense but can be called AI, and describes itself as a large language model trained by Google, capable of communicating and generating human-like text. It then asks how it can help Muhammad. Muhammad says he is learning about "chain" and exploring its uses. The AI responds enthusiastically, clarifies that "chain" could refer to blockchain technology, Markov chains, or supply chain management, and provides a brief overview of each, including potential applications. It then asks Muhammad to specify which type of "chain" he is most interested in


> Finished chain.


{'input': 'BufferWindowMemory stores last k messages and dropped previous messages',
 'history': 'The human introduces himself as Muhammad. The AI greets Muhammad, explains it doesn\'t have a name in the human sense but can be called AI, and describes itself as a large language model trained by Google, capable of communicating and generating human-like text. It then asks how it can help Muhammad. Muhammad says he is learning about "chain" and exploring its uses. The AI responds enthusiastically, clarifies that "chain" could refer to blockchain technology, Markov chains, or supply chain management, and provides a brief overview of each, including potential applications. It then asks Muhammad to specify which type of "chain" he is most interested in to provide more relevant information. Muhammad specifies he is looking into ConversationBufferMemory and ConversationWindowBufferMemory. The AI confirms Muhammad is exploring conversational AI and memory management, then provides detailed exp

In [31]:
summary_chain.invoke({"input": "What is my name"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces himself as Muhammad. The AI greets Muhammad, explains it doesn't have a name in the human sense but can be called AI, and describes itself as a large language model trained by Google, capable of communicating and generating human-like text. It then asks how it can help Muhammad. Muhammad says he is learning about "chain" and exploring its uses. The AI responds enthusiastically, clarifies that "chain" could refer to blockchain technology, Markov chains, or supply chain management, and provides a brief overview of each, including potential applications. It then asks Muhammad to specify which type of "chain" he is most interested in to provide more re

{'input': 'What is my name',
 'history': 'The human introduces himself as Muhammad. The AI greets Muhammad, explains it doesn\'t have a name in the human sense but can be called AI, and describes itself as a large language model trained by Google, capable of communicating and generating human-like text. It then asks how it can help Muhammad. Muhammad says he is learning about "chain" and exploring its uses. The AI responds enthusiastically, clarifies that "chain" could refer to blockchain technology, Markov chains, or supply chain management, and provides a brief overview of each, including potential applications. It then asks Muhammad to specify which type of "chain" he is most interested in to provide more relevant information. Muhammad specifies he is looking into ConversationBufferMemory and ConversationWindowBufferMemory. The AI confirms Muhammad is exploring conversational AI and memory management, then provides detailed explanations of ConversationBufferMemory (a running transcr

## Conversation Summary Memory with Runnable Message History

In [32]:
from langchain_core.messages import SystemMessage

In [33]:
class ConversationSummaryMessageHistory(BaseChatMessageHistory, BaseModel): 
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: GoogleGenerativeAI = Field(default_factory=GoogleGenerativeAI)
    
    def __init__(self, llm:GoogleGenerativeAI):
        super().__init__(llm=llm)
    
    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history
        """
        self.messages.extend(messages)
        # construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensuring to maintain "
                "as much relevant information as possible."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{messages}"
            )
        ])
        # format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=self.messages,
                messages=[x.content for x in messages]
            )
        )
        # replace the existing history with a single system summary message
        self.messages = [SystemMessage(content=new_summary)]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []
    

In [34]:
summary_chat_map = {}

def get_summary_chat_history(session_id: str, llm: GoogleGenerativeAI) -> ConversationSummaryMessageHistory:
    if session_id not in summary_chat_map:
        summary_chat_map[session_id] = ConversationSummaryMessageHistory(llm = llm)
    
    return summary_chat_map[session_id]

In [35]:
pipeline_csmh = (
        { 
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
        }
    | prompt_template
    | llm
)

pipeline_with_summary_history = RunnableWithMessageHistory(
    pipeline_csmh,
    get_session_history = get_summary_chat_history,
    input_messages_key = "query",
    history_messages_key = "history",
    history_factory_config = [
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=GoogleGenerativeAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        )
        
    ]
)

In [36]:
session_id = "csbm_123"

pipeline_with_summary_history.invoke(
    {
        "query": "Hi, my name is Muhammad"
    },
    config={
        "session_id": session_id, 
        "llm": llm
    }
)

"Hello Muhammad, it's nice to meet you! My name is Zeta. How can I help you today?"

In [37]:
summary_chat_map[session_id].messages

[SystemMessage(content='The conversation began with Muhammad introducing himself. Zeta, the AI, responded, introducing herself and offering assistance.', additional_kwargs={}, response_metadata={})]

In [38]:
pipeline_with_summary_history.invoke(
    {
        "query": "I'm researching different type of conversational memory"
    },
    config={
        "session_id": session_id, 
        "llm": llm
    }
)

"Okay, Muhammad. I understand you're researching different types of conversational memory. That's a fascinating area! To help me understand what you're looking for, could you tell me a bit more about what aspects you're interested in? For example, are you focusing on:\n\n*   **Specific types of memory architectures?** (e.g., episodic memory, semantic memory, working memory, long-term memory, etc.)\n*   **Different techniques for implementing conversational memory in AI systems?** (e.g., using embeddings, knowledge graphs, attention mechanisms, etc.)\n*   **The challenges and limitations of current conversational memory approaches?** (e.g., dealing with context length, maintaining consistency, handling ambiguity, etc.)\n*   **Applications of conversational memory in specific domains?** (e.g., customer service, education, healthcare, etc.)\n\nThe more information you can give me, the better I can assist you in your research. I can provide information, suggest resources, and help you brai

In [39]:
for msg in [
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]:
    pipeline_with_summary_history.invoke(
        {"query": msg},
        config={"session_id": session_id, "llm": llm}
    )

In [40]:
summary_chat_map[session_id].messages

[SystemMessage(content="The conversation began with Muhammad introducing himself to Zeta, the AI, stating he is researching different types of conversational memory. Zeta offered assistance and asked for clarification on the specific aspects of conversational memory Muhammad is interested in, such as memory architectures, implementation techniques, challenges, or applications in specific domains. Muhammad stated he has been looking at ConversationBufferMemory and ConversationBufferWindowMemory. Zeta confirmed that ConversationBufferMemory stores the entire conversation history as a single string, making it easy to access the full context but also presenting drawbacks as the conversation grows. Buffer window memory stores the last k messages, dropping the rest, which Zeta confirmed. Zeta then prompted Muhammad to consider the implications and potential issues of storing the entire conversation, and the trade-offs between the two approaches, also inquiring if he is considering other memo

In [41]:
    pipeline_with_summary_history.invoke(
        {"query": "what is my name again"},
        config={"session_id": session_id, "llm": llm}
    )

'Your name is Muhammad.'

### ConversationSummaryBufferMemory

Our final memory type acts as a combination of ConversationSummaryMemory and ConversationBufferMemory.


In [42]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=300,
    return_messages=True
)

csbm1_chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)


/tmp/ipykernel_2375/533772851.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [43]:
csbm1_chain.invoke({"input": "Hi, my name is James"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hi, my name is James
AI:

> Finished chain.


{'input': 'Hi, my name is James',
 'history': [HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi James! It's nice to meet you. My name is, well, I don't really *have* a name in the traditional sense. You can just call me AI. I'm a large language model, trained by Google. I'm excited to chat with you today! What's on your mind?", additional_kwargs={}, response_metadata={})],
 'response': "Hi James! It's nice to meet you. My name is, well, I don't really *have* a name in the traditional sense. You can just call me AI. I'm a large language model, trained by Google. I'm excited to chat with you today! What's on your mind?"}

In [44]:
for msg in [
    "I'm researching the different types of conversational memory.",
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]:
    csbm1_chain.invoke({"input": msg})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi James! It's nice to meet you. My name is, well, I don't really *have* a name in the traditional sense. You can just call me AI. I'm a large language model, trained by Google. I'm excited to chat with you today! What's on your mind?", additional_kwargs={}, response_metadata={})]
Human: I'm researching the different types of conversational memory.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific detail

### ConversationSummaryBufferMemory with RunnableWithMessageHistory

In [77]:
class ConversationSummaryBufferMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: GoogleGenerativeAI = Field(default_factory=GoogleGenerativeAI)
    k: int = Field(default_factory=int)

    def __init__(self, llm: GoogleGenerativeAI, k: int):
        super().__init__(llm=llm, k=k)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages and summarizing the messages that we
        drop.
        """
        existing_summary: SystemMessage | None = None
        old_messages: list[BaseMessage] | None = None
        # see if we already have a summary message
        if len(self.messages) > 0 and isinstance(self.messages[0], SystemMessage):
            print(">> Found existing summary")
            existing_summary = self.messages.pop(0)
        # add the new messages to the history
        self.messages.extend(messages)
        # check if we have too many messages
        if len(self.messages) > self.k:
            print(
                f">> Found {len(self.messages)} messages, dropping "
                f"oldest {len(self.messages) - self.k} messages."
            )
            # pull out the oldest messages...
            old_messages = self.messages[:self.k]
            # ...and keep only the most recent messages
            self.messages = self.messages[-self.k:]
        if old_messages is None:
            print(">> No old messages to update summary with")
            # if we have no old_messages, we have nothing to update in summary
            return
        # construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensuring to maintain "
                "as much relevant information as possible."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{old_messages}"
            )
        ])
        # format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=existing_summary,
                old_messages=old_messages
            )
        )
        print(f">> New summary: {new_summary}")
        # prepend the new summary to the history
        self.messages = [SystemMessage(content=new_summary)] + self.messages

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [81]:
chat_map_csbm_1 = {}
def get_chat_history_csbm_1(session_id: str, llm: GoogleGenerativeAI, k: int) -> ConversationSummaryBufferMessageHistory:
    
    if session_id not in chat_map_csbm_1:
        print(f"session not exists {session_id}")
        # if session ID doesn't exist, create a new chat history
        chat_map_csbm_1[session_id] = ConversationSummaryBufferMessageHistory(llm=llm, k=k)
    # return the chat history
    return chat_map_csbm_1[session_id]

In [82]:
pipeline_csbmh = (
        { 
        "query": lambda x: x["query"],
        "history": lambda x: x["history"]
        }
    | prompt_template
    | llm
)


pipeline_with_history_csbm_1 = RunnableWithMessageHistory(
    pipeline_csbmh,
    get_session_history=get_chat_history_csbm_1,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=GoogleGenerativeAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=llm,
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        )
    ]
)

In [83]:
csbm_session_id = "csbm_123"
pipeline_with_history_csbm_1.invoke(
        {"query": "Hi, my name is Muhammad"},
        config={"session_id": csbm_session_id, "llm": llm, "k": 4}
)

session not exists csbm_123
>> No old messages to update summary with


"Hello Muhammad, it's nice to meet you! My name is Zeta. How can I help you today?"

In [84]:
for i, msg in enumerate([
    "I'm researching the different types of conversational memory.",
    "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory.",
    "Buffer memory just stores the entire conversation",
    "Buffer window memory stores the last k messages, dropping the rest."
]):
    print(f"---\nMessage {i+1}\n---\n")
    pipeline_with_history_csbm_1.invoke(
        {"query": msg},
        config={"session_id": csbm_session_id, "llm": llm, "k": 4}
    )

---
Message 1
---

>> No old messages to update summary with
---
Message 2
---

>> Found 6 messages, dropping oldest 2 messages.
>> New summary: Muhammad is researching conversational memory and has introduced himself to Zeta, the AI assistant. Zeta is offering to help and has asked Muhammad to clarify his research interests, providing several examples of possible areas of focus within conversational memory.
---
Message 3
---

>> Found existing summary
>> Found 6 messages, dropping oldest 2 messages.
>> New summary: content='Muhammad is researching conversational memory and has introduced himself to Zeta, the AI assistant. Zeta is helping Muhammad by explaining different types of conversational memory, specifically focusing on ConversationBufferMemory and ConversationBufferWindowMemory. Zeta detailed their functionalities, pros, cons, use cases, and key differences, including how they store information, their scalability, context retention, computational cost, and complexity. Zeta also